In [14]:
import sys
print(sys.version)
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

2.7.12rc1 (v2.7.12rc1:13912cd1e7e8, Jun 11 2016, 15:32:34) 
[GCC 4.2.1 (Apple Inc. build 5666) (dot 3)]


In [15]:
spark = SparkSession.builder.master("local[*]").appName("SENG474-HD").getOrCreate()

In [16]:
df = spark.read.load('/Users/Matthew/Downloads/data_Q1_2017/2017-01-01.csv', 
                     format='com.databricks.spark.csv', 
                     header='true', 
                     inferSchema='true')

In [17]:
#print("Number of data points:", str(df.count()))

In [58]:
df.columns

['date',
 'serial_number',
 'model',
 'capacity_bytes',
 'failure',
 'smart_1_normalized',
 'smart_1_raw',
 'smart_2_normalized',
 'smart_2_raw',
 'smart_3_normalized',
 'smart_3_raw',
 'smart_4_normalized',
 'smart_4_raw',
 'smart_5_normalized',
 'smart_5_raw',
 'smart_7_normalized',
 'smart_7_raw',
 'smart_8_normalized',
 'smart_8_raw',
 'smart_9_normalized',
 'smart_9_raw',
 'smart_10_normalized',
 'smart_10_raw',
 'smart_11_normalized',
 'smart_11_raw',
 'smart_12_normalized',
 'smart_12_raw',
 'smart_13_normalized',
 'smart_13_raw',
 'smart_15_normalized',
 'smart_15_raw',
 'smart_22_normalized',
 'smart_22_raw',
 'smart_183_normalized',
 'smart_183_raw',
 'smart_184_normalized',
 'smart_184_raw',
 'smart_187_normalized',
 'smart_187_raw',
 'smart_188_normalized',
 'smart_188_raw',
 'smart_189_normalized',
 'smart_189_raw',
 'smart_190_normalized',
 'smart_190_raw',
 'smart_191_normalized',
 'smart_191_raw',
 'smart_192_normalized',
 'smart_192_raw',
 'smart_193_normalized',
 'sma

In [19]:
#print(df.show(10, True))

In [20]:
from pyspark.mllib.regression import LabeledPoint
def transform_data(data):
    prediction_col = "failure"
    columns = []#["serial_number", "model", "capacity_bytes"]
    # Add Smart columns.
    for i in range(1, 255):
        test_col = "smart_" + str(i) + "_normalized"
        try:
            data[test_col]
            columns.append(test_col)
        except:
            pass
    return LabeledPoint(float(data[prediction_col]), [0.0 if data[i] is None else data[i] for i in columns])

In [21]:
df_model = df.rdd.map(transform_data)
#print(df_model.count())

In [22]:
(trainingData, testData) = df_model.randomSplit([0.7, 0.3])

In [23]:
from pyspark.mllib.tree import DecisionTree
#dt = DecisionTree.trainClassifier(trainingData, numClasses=2,categoricalFeaturesInfo={})

In [24]:
#predictions = dt.predict(testData.map(lambda x: x.features))
#labels_and_predictions = testData.map(lambda lp: lp.label).zip(predictions)
#testErr = float(labels_and_predictions.filter(lambda lp: lp[0] != lp[1]).count()) / float(testData.count())
#print("Test Error: ", str(testErr))

In [95]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql import DataFrame
from pyspark.ml.linalg import Vectors


def transform_data2(data):
    prediction_col = "failure"
    columns = [] #["serial_number", "model", "capacity_bytes"]
    # Add Smart columns.
    
    if data[prediction_col] is None:
        print(data)
    
    for i in range(1, 255):
        test_col = "smart_" + str(i) + "_normalized"
        try:
            data[test_col]
            columns.append(test_col)
        except:
            pass
    return (float(data[prediction_col]), Vectors.dense(*[data[i] for i in columns]))

In [98]:
#labels_and_predictions.show(10, False)
df.limit(1).fillna(0.0).rdd.map(transform_data2).toDF(["label", "features"]).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[100.0,135.0,127....|
+-----+--------------------+



In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


(trainingData, testData) = df.fillna(0.0).rdd.map(transform_data2).toDF(["label", "features"]).randomSplit([0.7, 0.3])
numFolds = 5

rf = RandomForestClassifier(maxMemoryInMB=1024)
evaluator = MulticlassClassificationEvaluator() # + other params as in Scala

#.addGrid(rf.impurity, ["variance", "gini"])\
paramGrid = ParamGridBuilder()\
    .addGrid(rf.maxDepth, [2, 4, 6, 8, 10]) \
    .addGrid(rf.numTrees, [1, 10, 20, 30])\
    .build()

pipeline = Pipeline(stages=[rf])

crossval = CrossValidator(
    estimator=pipeline,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    numFolds=numFolds)

model = crossval.fit(trainingData)
predictions = model.predict(testData.map(lambda x: x.features))
labels_and_predictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = float(labels_and_predictions.filter(lambda lp: lp[0] != lp[1]).count()) / float(testData.count())
print("Random ForestTest Error: ", str(testErr))